In [13]:
import copy
import datetime
import os
import sys

sys.path.append("..")

from dotenv import load_dotenv

load_dotenv("../common/.env_development")

from contextlib import contextmanager

from common import config
from common.controllers.ml_controller import MLController
from google.cloud import storage
from google.cloud import datastore

@contextmanager
def prod_credentials():
    try:
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.environ[
            "CREDENTIALS_PROD_CONTAINER_PATH"
        ]
        yield
    finally:
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.environ[
            "CREDENTIALS_DEV_CONTAINER_PATH"
        ]

In [14]:
def delete_model(bucket_name, blob_name):
    """Deletes a blob from the bucket."""

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()

    print("Blob {} deleted.".format(blob_name))


def check_if_model_exists(body):
    time_range_start = datetime.datetime.strptime(body['training_data']['time_range_start'], '%Y-%m-%dT%H:%M:%SZ')
    time_range_end = datetime.datetime.strptime(body['training_data']['time_range_end'], '%Y-%m-%dT%H:%M:%SZ')
    client = datastore.Client()
    query = client.query(kind='MLModelMetadata')
    query.add_filter('algorithm', '=', body['algorithm'])
    query.add_filter('optimization_metric', '=', body['optimization_metric'])
    query.add_filter('node', '=', body['training_data']['node'])
    query.add_filter('previous_time_range_start', '=', time_range_start)
    query.add_filter('previous_time_range_end', '=', time_range_end)
    if body['target_frequency'] == '1h':
        query.add_filter('target_frequency', '=', 'P0DT1H0M0S')
    if body['horizon'] == '1d':
        query.add_filter('horizon', '=', 'P1DT0H0M0S')
    elif body['horizon'] == '7d':
        query.add_filter('horizon', '=', 'P7DT0H0M0S')
    return list(query.fetch())

In [18]:
time_range_start = '2020-08-14T14:00:00Z'
time_range_end = '2020-08-22T00:00:00Z'
node = 100
opt_metric = 'r2'
horizon = '1d'

request_body = {
                'algorithm': 'autoarima',
                'optimization_metric': opt_metric,
                'horizon': horizon,
                'training_data': {
                  'node': node,
                  'endogenous': 'energy_usage',
                  'time_range_start': time_range_start,
                  'time_range_end': time_range_end},
                'target_frequency': '1h'}

In [19]:
tested_algorithms = ['autoarima', 'autosarima', 'arima', 'sarima', 'decision_tree', 'random_forest',
                     'linear_regression', 'lasso', 'svm', 'boosted_trees', 'mlp']
params = {"arima": {"order": [[0, 1, 2]]}, "sarima": {"order": [[0, 1, 2]]}}
mlcontroller = MLController()
tasks_ids = []

In [20]:
for algorithm in tested_algorithms:
    new_request_body = copy.deepcopy(request_body)
    if algorithm in params.keys():
        new_request_body['params'] = params[algorithm]
    new_request_body['algorithm'] = algorithm
    list_of_same_models = check_if_model_exists(new_request_body)
    if len(list_of_same_models) == 0:
        response = mlcontroller.create_and_train_model(new_request_body)
    else:
        response = mlcontroller.create_and_train_model(new_request_body)
        print(f'Liczba modeli o tych samych parametrach: {len(list_of_same_models)}')
    print(response.status_code, response.text)
    tasks_ids.append(response.json()['task_id'])


Creating model with request body {'algorithm': 'autoarima', 'optimization_metric': 'r2', 'horizon': '1d', 'training_data': {'node': 100, 'endogenous': 'energy_usage', 'time_range_start': '2020-08-14T14:00:00Z', 'time_range_end': '2020-08-22T00:00:00Z'}, 'target_frequency': '1h'}...
202 {"task_id":"93d781d9-bfe7-45af-b4c9-760b65325bc7"}

Creating model with request body {'algorithm': 'autosarima', 'optimization_metric': 'r2', 'horizon': '1d', 'training_data': {'node': 100, 'endogenous': 'energy_usage', 'time_range_start': '2020-08-14T14:00:00Z', 'time_range_end': '2020-08-22T00:00:00Z'}, 'target_frequency': '1h'}...
202 {"task_id":"567ef7d0-078a-4f1f-b002-2ce57eba3cc2"}

Creating model with request body {'algorithm': 'arima', 'optimization_metric': 'r2', 'horizon': '1d', 'training_data': {'node': 100, 'endogenous': 'energy_usage', 'time_range_start': '2020-08-14T14:00:00Z', 'time_range_end': '2020-08-22T00:00:00Z'}, 'target_frequency': '1h', 'params': {'order': [[0, 1, 2]]}}...
202 {"ta

In [21]:
model_ids = []
for task_id in tasks_ids:
    task_response = mlcontroller.wait_for_task(task_id, 600)
    print(task_response.status_code, task_response.text)
    if task_response.json()["status"] == 'SUCCESS':
        model_ids.append(task_response.json()["result"]["model_id"])

Waiting for task of id 93d781d9-bfe7-45af-b4c9-760b65325bc7 to finish
Waiting. Status: STARTED
Waiting. Status: STARTED
Waiting. Status: STARTED
Task finished!
200 {"message":null,"result":{"metrics":{"mae":0.2762979108166336,"mape":1.4427822642669659,"me":1.1577611782053419,"mse":0.10494549718388141,"r2":0.24218155954496579,"rmae":0.5256404767677557,"rmse":0.3239529243329676},"model_id":"937cfba9-12f8-4345-80e1-248e9a24b68b","selected_hyperparameters":{}},"status":"SUCCESS"}

Waiting for task of id 567ef7d0-078a-4f1f-b002-2ce57eba3cc2 to finish
Task finished!
200 {"message":null,"result":{"metrics":{"mae":0.2746126766194764,"mape":1.4447572987228787,"me":1.1578433920132012,"mse":0.10458254746186284,"r2":0.24480244371517235,"rmae":0.5240349956057099,"rmse":0.32339225015739453},"model_id":"81da0241-5403-404f-9479-045b984a02c5","selected_hyperparameters":{}},"status":"SUCCESS"}

Waiting for task of id 709b6e3a-c0e9-4559-841c-78373c7882ef to finish
Waiting. Status: STARTED
Task finished!


In [ ]:
for model_id in model_ids:
    delete_model('pwrability-development-ml-models-metadata', f'model_{model_id}.json')

Blob model_937cfba9-12f8-4345-80e1-248e9a24b68b.json deleted.
Blob model_81da0241-5403-404f-9479-045b984a02c5.json deleted.
Blob model_56e8a93b-5648-4624-890b-2ead7758f557.json deleted.
Blob model_1aad2f6a-23fb-4f82-b678-102841cf2cc5.json deleted.
Blob model_34c5b73a-f45f-4e71-a032-f364882b254c.json deleted.
Blob model_3319b9a6-1263-4b29-89a2-fb24db1646cd.json deleted.
Blob model_dd676923-4ee2-447c-87cd-be6142498321.json deleted.
Blob model_52f9cf10-aaa3-4d9e-8c72-3c5f876b370c.json deleted.
Blob model_a8e475bf-9c51-4fe5-8965-4473444297cd.json deleted.
Blob model_a5c7a08e-7e1e-4623-b9f2-38a13bec2f0e.json deleted.


Creating model with request body {'algorithm': 'sarima', 'params': {'order': [[0, 1, 2]]}, 'optimization_metric': 'r2', 'horizon': '1d', 'training_data': {'node': 100, 'endogenous': 'energy_usage', 'time_range_start': '2020-08-12T00:00:00Z', 'time_range_end': '2020-08-22T00:00:00Z'}, 'target_frequency': '1h'}...


<Response [202]>